In [3]:
from bs4 import BeautifulSoup
import numpy as np
import os
import pymorphy2
import re
from stop_words import get_stop_words
import pickle

AllowedWords = '[а-я]+'

wordNormal = pymorphy2.MorphAnalyzer()
stop_words = {wordNormal.normal_forms(i)[0] for i in get_stop_words('ru')}


MODEL_NAME = 'finaleModel.bin'

#This is the part where all docs and discussions get cleaned and stored
#Also test and train txt was created here

fileMap = dict()

for num,name in enumerate(os.listdir("./Dirty/Data/")):
    fileMap[num] = name
    with open("./Dirty/Data/"+name, 'r') as doc:
        soup = BeautifulSoup(doc.read(), 'html.parser')
        with open("./Clean/Data/"+str(num)+'.txt', 'w') as clean:
            clean.write(' '.join(soup.getText(' ').strip().split()))
            
text = ''
for name in os.listdir("./Clean/Data/"):
    try:
        with open("./Clean/Data/"+name, 'r',) as doc:
            for word in re.findall(AllowedWords, doc.read().lower()):
                normal = wordNormal.normal_forms(word)[0]
                text += str(normal) + ' '
            text += '\n'
    except:
        print("haha")
with open('./Technical/test.txt', 'w') as file:
        file.write(text)
        
for num, name in enumerate(os.listdir("./Dirty/Discussions/")):
    with open("./Dirty/Discussions/"+name, 'r') as doc:
        soup = BeautifulSoup(doc.read(), 'html.parser')
        with open("./Clean/Discussions/"+str(num)+'.txt', 'w') as clean:
            clean.write(' '.join(soup.getText(' ').strip().split()))
            
text = ''
for name in os.listdir("./Clean/Discussions/"):
    try:
        with open("./Clean/Discussions/"+name, 'r') as doc:
            for word in re.findall(AllowedWords, doc.read().lower()):
                normal = wordNormal.normal_forms(word)[0]
                text += str(normal) + ' '
            text += '\n'
    except:
        print("haha")
with open('./Technical/train.txt', 'w') as file:
    file.write(text)


#This is part where model was created
    
command = """./sent2vec/fasttext sent2vec -input \
    ./Technical/train.txt -output ./Model/{} -minCount 2 -dim 700\
        -epoch 9 -lr 0.15 -wordNgrams 2 -loss ns -neg 10 -thread 2\
            -t 0.000005 -dropoutK 4 -minCountLabel 10""".format(MODEL_NAME)
os.system(command)

#Here we create vectors for each doc
command = """./sent2vec/fasttext print-sentence-vectors ./Model/{} < ./Technical/test.txt""".format(MODEL_NAME)
output = os.popen(command).readlines()

#Cleaned all docs
clearVec = dict()
with open('./Technical/test.txt', 'r') as test:
    text = test.readlines()
    for num, i in enumerate(output):
        clearVec[(text[num].strip())] = [float(j) for j in i.strip().split()]

#Let's store our clearVec with pickle

with open('./Technical/clearVec.pickle', 'wb') as file:
    pickle.dump(clearVec, file)
    
#And dont forgot our file mapping file

with open('./Technical/fileMapping.pickle', 'wb') as file:
    pickle.dump(fileMap, file)


In [4]:
fileMap

{0: 'DOC-10091.html',
 1: 'DOC-10092.html',
 2: 'DOC-10093.html',
 3: 'DOC-10094.html',
 4: 'DOC-10392.html',
 5: 'DOC-10422.html',
 6: 'DOC-105943.html',
 7: 'DOC-106600.html',
 8: 'DOC-12204.html',
 9: 'DOC-12223.html',
 10: 'DOC-1464.html',
 11: 'DOC-1473.html',
 12: 'DOC-15172.html',
 13: 'DOC-1554.html',
 14: 'DOC-1555.html',
 15: 'DOC-1556.html',
 16: 'DOC-1571.html',
 17: 'DOC-1572.html',
 18: 'DOC-1578.html',
 19: 'DOC-15821.html',
 20: 'DOC-17395.html',
 21: 'DOC-17643.html',
 22: 'DOC-18044.html',
 23: 'DOC-18078.html',
 24: 'DOC-1902.html',
 25: 'DOC-1936.html',
 26: 'DOC-20926.html',
 27: 'DOC-21430.html',
 28: 'DOC-21638.html',
 29: 'DOC-21641.html',
 30: 'DOC-22163.html',
 31: 'DOC-23526.html',
 32: 'DOC-27881.html',
 33: 'DOC-2837.html',
 34: 'DOC-33678.html',
 35: 'DOC-33688.html',
 36: 'DOC-33999.html',
 37: 'DOC-34224.html',
 38: 'DOC-3660.html',
 39: 'DOC-36664.html',
 40: 'DOC-36815.html',
 41: 'DOC-38728.html',
 42: 'DOC-39159.html',
 43: 'DOC-39297.html',
 44: 'DO